In [2]:
# ============================
# Imports & Environment Setup
# ============================
import sys
import subprocess
import importlib
import warnings
import os
import json
import io
import time
import itertools
import heapq
import random
from collections import defaultdict
from math import isfinite

# Utility: ensure packages are installed (pip)
def ensure_packages(pkgs):
    """
    pkgs: list of package names as passed to pip (strings).
    This tries to import the package; if import fails, pip installs it.
    """
    for pkg in pkgs:
        try:
            importlib.import_module(pkg.replace("-", "_"))
        except ImportError:
            print(f"Installing {pkg} ...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

# Install / ensure commonly used packages for this notebook
ensure_packages([
    "requests",
    "folium",
    "ortools",
    "qiskit",
    "qiskit-aer",
    "qiskit-algorithms",
    "qiskit-optimization",
    "matplotlib",
    "numpy"
])

# ---- Standard library / common imports ----
import numpy as np
import matplotlib.pyplot as plt
import requests
import folium
from folium import Popup
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# ---- Qiskit imports (robust) ----
from qiskit_algorithms.optimizers import NELDER_MEAD
from qiskit import QuantumCircuit, transpile
from qiskit_aer.backends.aer_simulator import AerSimulator
try:
    simulator = AerSimulator(device='GPU')
    print("GPU found and ready.")
except Exception as e:
    print(f"GPU not found. Error: {e}")
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import SparsePauliOp

print("All packages loaded successfully!")


GPU found and ready.
All packages loaded successfully!


In [3]:
# =============================================================
# SECTION 1: DATA LOADING AND DISTANCE MATRIX GENERATION
# =============================================================

# Problem data
data = {
    "problem_description": {
        "title": "Emergency Patient Transportation System",
        "objective": "Minimize total distance traveled while transporting all patients to hospital",
        "constraints": {"vehicles": 1, "capacity_per_stop": 1, "stops_per_trip": 3, "total_patients": 5}
    },
    "locations": {
        "hospital": {"name": "Central Hospital", "coordinates": {"latitude": 29.99512653425452, "longitude": 31.68462840171934}},
        "patients": [
            {"id": "DT", "name": "Patient DT", "coordinates": {"latitude": 30.000417586266437, "longitude": 31.73960813272627}},
            {"id": "GR", "name": "Patient GR", "coordinates": {"latitude": 30.011344405285193, "longitude": 31.747827362371993}},
            {"id": "R2", "name": "Patient R2", "coordinates": {"latitude": 30.030388325206854, "longitude": 31.669231198639675}},
            {"id": "R3_2", "name": "Patient R3_2", "coordinates": {"latitude": 30.030940768851426, "longitude": 31.688371339937028}},
            {"id": "IT", "name": "Patient IT", "coordinates": {"latitude": 30.01285635906825, "longitude": 31.693811715848444}}
        ]
    }
}

hospital = data["locations"]["hospital"]["coordinates"]
patients = data["locations"]["patients"]
ids = [p["id"] for p in patients]
coord_map = {"H": (hospital["latitude"], hospital["longitude"])}
for p in patients:
    coord_map[p["id"]] = (p["coordinates"]["latitude"], p["coordinates"]["longitude"])

# Helper functions for distance matrix
def to_lonlat_list(keys):
    return [(coord_map[k][1], coord_map[k][0]) for k in keys]

def fetch_osrm_table_matrix(keys, timeout=20):
    coords = to_lonlat_list(keys)
    coord_str = ";".join(f"{lon:.6f},{lat:.6f}" for lon, lat in coords)
    url = f"http://router.project-osrm.org/table/v1/driving/{coord_str}?annotations=distance"
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    js = r.json()
    if "distances" not in js:
        raise RuntimeError("OSRM response missing 'distances'")
    return [[(v / 1000.0 if v is not None else float('inf')) for v in row] for row in js["distances"]]

def build_distance_matrix_fallback():
    def euclidean_distance(coord1, coord2):
        return np.sqrt((coord2[0] - coord1[0])**2 + (coord2[1] - coord1[1])**2) * 111
    points = ["H"] + ids
    dist = {}
    for a in points:
        dist[a] = {b: euclidean_distance(coord_map[a], coord_map[b]) for b in points}
    return dist

# Try to get road distances, fallback to Euclidean
try:
    print("Attempting to fetch road distances from OSRM...")
    points = ["H"] + ids
    api_mat = fetch_osrm_table_matrix(points)
    dist = {a: {b: float(api_mat[i][j]) for j, b in enumerate(points)} for i, a in enumerate(points)}
    print("Successfully obtained road distances from OSRM!")
except Exception as e:
    print(f"OSRM failed: {e}. Using Euclidean distance approximation...")
    dist = build_distance_matrix_fallback()

# Display distance matrix
print("\nDistance Matrix (km):")
points = ["H"] + ids
header = " " * 6 + "".join(f"{p:>8}" for p in points)
print(header)
print("-" * len(header))
for a in points:
    row_str = f"{a:>6}: " + "".join(f"{dist[a][b]:8.2f}" for b in points)
    print(row_str)


Attempting to fetch road distances from OSRM...
Successfully obtained road distances from OSRM!

Distance Matrix (km):
             H      DT      GR      R2    R3_2      IT
------------------------------------------------------
     H:     0.00    8.63   11.50    9.45   10.85    9.67
    DT:    14.19    0.00    2.36   10.92    9.24    9.43
    GR:    17.78    7.75    0.00   11.81   10.12   10.48
    R2:    11.86   19.67   15.66    0.00   11.57   11.54
  R3_2:     7.34   12.17   10.05    4.07    0.00    5.93
    IT:     9.27    9.40   12.27    7.32    8.72    0.00


In [4]:

# =============================================================
## SECTION 2: CLASSICAL SOLVER (OR-TOOLS)
# =============================================================

def solve_classical_vrp():
    """
    Solves the Vehicle Routing Problem using Google OR-Tools to get a
    classical benchmark solution.
    """
    print("\n=== CLASSICAL SOLVER (OR-TOOLS) ===")

    # Create the routing index manager.
    # Nodes are indexed 0 to N-1. 'H' is depot (index 0).
    nodes = ["H"] + ids
    manager = pywrapcp.RoutingIndexManager(len(nodes), 2, 0) # 2 vehicles/trips, depot at node 0

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        from_node_idx = manager.IndexToNode(from_index)
        to_node_idx = manager.IndexToNode(to_index)
        return int(dist[nodes[from_node_idx]][nodes[to_node_idx]] * 1000) # Use integer meters

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add constraint: each trip can have at most 3 patient stops + 1 hospital return = 4 legs
    routing.AddDimension(
        routing.RegisterUnaryTransitCallback(lambda index: 1),
        0,  # no slack
        4,  # vehicle capacity
        True,
        "stops"
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        total_distance = solution.ObjectiveValue() / 1000.0
        full_tour = []
        for vehicle_id in range(2):
            index = routing.Start(vehicle_id)
            route = []
            while not routing.IsEnd(index):
                node_idx = manager.IndexToNode(index)
                route.append(nodes[node_idx])
                index = solution.Value(routing.NextVar(index))
            # The route implicitly ends at the depot, so add it manually
            route.append(nodes[0])
            if len(route) > 2: # Only add non-empty trips
                 full_tour.extend(route if not full_tour else route[1:])
        return full_tour, total_distance
    else:
        print("Classical solver found no solution.")
        return None, float('inf')

# Run classical solver to get the benchmark
classical_tour, classical_distance = solve_classical_vrp()
if classical_tour:
    print(f"\nClassical Solution Found:")
    print(f"  Optimal Tour: {' -> '.join(classical_tour)}")
    print(f"  Total Distance: {classical_distance:.2f} km")


=== CLASSICAL SOLVER (OR-TOOLS) ===

Classical Solution Found:
  Optimal Tour: H -> IT -> R2 -> H -> DT -> GR -> R3_2 -> H
  Total Distance: 57.31 km


In [ ]:
import numpy as np
from numpy import isfinite

# --- Mock Data for Self-Contained Execution ---
ids = ['DT', 'GR', 'R2', 'R3 2', 'IT']
dist = {
    'H': {'DT': 8.628, 'GR': 11.496, 'R2': 9.445, 'R3 2': 10.852, 'IT': 9.672, 'H': 0.0},
    'DT': {'H': 14.194, 'GR': 2.361, 'R2': 10.922, 'R3 2': 9.238, 'IT': 9.430, 'DT': 0.0},
    'GR': {'H': 17.785, 'GR': 0.0, 'DT': 7.745, 'R2': 11.808, 'R3 2': 10.124, 'IT': 10.478},
    'R2': {'H': 11.864, 'DT': 19.672, 'GR': 15.661, 'R2': 0.0, 'R3 2': 11.572, 'IT': 11.538},
    'R3 2': {'H': 7.343, 'DT': 12.172, 'GR': 10.053, 'R2': 4.071, 'R3 2': 0.0, 'IT': 5.931},
    'IT': {'H': 9.269, 'DT': 9.398, 'GR': 12.266, 'R2': 7.318, 'R3 2': 8.725, 'IT': 0.0}
}
# =============================================================
# RECONSTRUCTED QUBO FORMULATION (30 QUBITS)
# =============================================================

def create_reconstructed_qubo_matrix():
    """
    Constructs a 30x30 QUBO matrix for the ambulance routing problem
    based on the specific constraints outlined in the presentation slides.
    """
    print("\n=== RECONSTRUCTED QUBO FORMULATION (30 QUBITS) ===")

    nodes = ["H"] + ids
    n = len(nodes) # Should be 6 (1 Hospital + 5 Patients)
    
    # Create a mapping from a path (i, j) to a unique qubit index
    var_map = {}
    qubit_idx = 0
    for i in range(n):
        for j in range(n):
            if i != j:
                var_map[(i, j)] = qubit_idx
                qubit_idx += 1

    num_qubits = qubit_idx
    print(f"Total variables (qubits): {num_qubits}")
    if num_qubits != 30:
        print(f"Warning: Expected 30 qubits for 6 nodes, but got {num_qubits}.")

    Q = np.zeros((num_qubits, num_qubits))

    # --- Objective Function: Minimize total travel distance ---
    # This is the linear part of the QUBO, placed on the diagonal of Q.
    for i in range(n):
        for j in range(n):
            if i != j:
                idx = var_map[(i, j)]
                Q[idx, idx] += dist[nodes[i]][nodes[j]]

    # --- Penalty Parameter (P) ---
    # A large penalty is crucial for enforcing constraints. We'll use a value
    # larger than the maximum possible valid path distance.
    total_dist = sum(dist[i][j] for i in dist for j in dist[i] if isfinite(dist[i][j]))
    P = total_dist * 1.4
    print(f"Using penalty parameter P: {P:.2f}")

    # ========================================================================
    # --- CONSTRAINT IMPLEMENTATION AS PER SLIDES 17-20 ---
    # ========================================================================
    
    # ========================================================================
    # --- REPLACEMENT FOR "Constraint 1: Enter-leave" ---
    # ========================================================================
    
    # --- Constraint 1: "Enter-leave" (Flow Conservation) ---
    # For each patient, the number of incoming edges must equal the number of outgoing edges.
    # Penalty: P * (sum_in x_in - sum_out x_out)^2 = 0 for each patient j.
    # for j in range(1, n): # For each patient node
    #     incoming_vars = [var_map[(i, j)] for i in range(n) if i != j]
    #     outgoing_vars = [var_map[(j, k)] for k in range(n) if k != j]
    
    #     # Add diagonal terms: P * (+1)^2 for incoming, P * (-1)^2 for outgoing
    #     for idx in incoming_vars + outgoing_vars:
    #         Q[idx, idx] += P
    
    #     # Add off-diagonal terms for pairs of incoming variables: P * 2 * (+1) * (+1) = 2P
    #     for i, idx1 in enumerate(incoming_vars):
    #         for idx2 in incoming_vars[i+1:]:
    #             a, b = sorted((idx1, idx2))
    #             Q[a, b] += 2 * P
        
    #     # Add off-diagonal terms for pairs of outgoing variables: P * 2 * (-1) * (-1) = 2P
    #     for i, idx1 in enumerate(outgoing_vars):
    #         for idx2 in outgoing_vars[i+1:]:
    #             a, b = sorted((idx1, idx2))
    #             Q[a, b] += 2 * P
    
    #     # Add off-diagonal terms for one incoming and one outgoing: P * 2 * (+1) * (-1) = -2P
    #     for idx_in in incoming_vars:
    #         for idx_out in outgoing_vars:
    #             a, b = sorted((idx_in, idx_out))
    #             Q[a, b] -= 2 * P
    for j in range(1, n): # For each patient node
        incoming_vars = [var_map[(i, j)] for i in range(n) if i != j]

        # Linear (diagonal) terms: -P * sum_i x_ij
        for idx in incoming_vars:
            Q[idx, idx] += -P
        
        # Quadratic cross terms: +2P * sum_{i<k} x_ij * x_kj
        for idx1_pos, idx1 in enumerate(incoming_vars):
            for idx2 in incoming_vars[idx1_pos + 1:]:
                a, b = (idx1, idx2) if idx1 < idx2 else (idx2, idx1)
                Q[a, b] += 2.0 * P
        
    # --- Constraint 2: Exactly 2 Trips from the Hospital ---
    # The number of outgoing edges from the hospital must be exactly 2.
    # We also enforce that the number of incoming edges is 2 to complete the trips.
    # Penalty: P * (sum_j x_0j - 2)^2 = 0 and P * (sum_i x_i0 - 2)^2 = 0
    for direction in ["outgoing", "incoming"]:
        if direction == "outgoing":
            # sum_{j>0} x_0j = 2
            edge_vars = [var_map[(0, j)] for j in range(1, n)]
        else: # incoming
            # sum_{i>0} x_i0 = 2
            edge_vars = [var_map[(i, 0)] for i in range(1, n)]
        
        # From (sum - 2)^2 = sum^2 - 4*sum + 4
        # Linear terms: (1-4) * P = -3P
        for idx in edge_vars:
            Q[idx, idx] -= 3 * P
        
        # Quadratic terms: 2P
        for i, idx1 in enumerate(edge_vars):
            for idx2 in edge_vars[i+1:]:
                a, b = sorted((idx1, idx2))
                Q[a, b] += 2 * P

    # --- Constraint 3: No Little Loops (Subtour Elimination) ---
    # Disallow taking only 1 patient in a trip (e.g., H -> P1 -> H).
    # This is achieved by penalizing the co-occurrence of x_0j and x_j0.
    # Penalty: P * x_0j * x_j0 for each patient j.
    # for j in range(1, n): # For each patient
    #     idx_0j = var_map[(0, j)]
    #     idx_j0 = var_map[(j, 0)]
    #     a, b = sorted((idx_0j, idx_j0))
    #     Q[a, b] += P

    # --- Constraint 4: Visit All Patient Nodes  ---
    # Every patient must have exactly one incoming edge.
    # Penalty: P * (sum_i x_ij - 1)^2 = 0 for each patient j.
    for j in range(1, n): # For each patient node
        incoming_vars = [var_map[(i, j)] for i in range(n) if i != j]

        # From (sum - 1)^2 = sum^2 - 2*sum + 1
        # Linear terms: (1-2) * P = -P
        for idx in incoming_vars:
            Q[idx, idx] -= P
        
        # Quadratic terms: 2P
        for i, idx1 in enumerate(incoming_vars):
            for idx2 in incoming_vars[i+1:]:
                a, b = sorted((idx1, idx2))
                Q[a, b] += 2 * P

    print(f"QUBO matrix shape: {Q.shape}")
    print(f"QUBO matrix non-zero elements: {np.count_nonzero(Q)}")

    return Q, var_map, nodes

# --- Execute the function ---
Q_matrix, variable_map, node_list = create_reconstructed_qubo_matrix()


=== RECONSTRUCTED QUBO FORMULATION (30 QUBITS) ===
Total variables (qubits): 30
Using penalty parameter P: 435.44
QUBO matrix shape: (30, 30)
QUBO matrix non-zero elements: 100


In [ ]:
# =============================================================
# SECTION 3: FULL QUBO FORMULATION (30 QUBITS)
# =============================================================

def create_full_qubo_matrix():
    print("\n=== FULL QUBO FORMULATION (30 QUBITS) ===")

    nodes = ["H"] + ids
    n = len(nodes)
    # The problem has 5 patients and 1 hospital, for a total of 6 nodes.
    # The slides' approach assumes 2 trips, which reduces the number of qubits to 30.
    # We use a full variable set x_ij for all i,j in nodes, where i != j.

    # Create variable mapping x_ij -> qubit index
    var_map = {}
    qubit_idx = 0
    for i in range(n):
        for j in range(n):
            if i != j:
                var_map[f"x_{i}_{j}"] = qubit_idx
                var_map[(i, j)] = qubit_idx
                qubit_idx += 1

    num_qubits = qubit_idx
    print(f"Total variables (qubits): {num_qubits}") # Should be 6 * 5 = 30
    if num_qubits != 30:
        print("Warning: The number of qubits is not 30. Please check the 'ids' variable.")

    Q = np.zeros((num_qubits, num_qubits))

    # --- Objective Function: Minimize total travel distance ---
    # Equation: min sum_i,j d_ij * x_ij
    # The linear term d_ij * x_ij becomes a diagonal element Q[idx, idx].
    for i in range(n):
        for j in range(n):
            if i != j:
                qubit_idx = var_map[(i, j)]
                Q[qubit_idx, qubit_idx] += dist[nodes[i]][nodes[j]]

    # --- CONSTRAINTS ---
    # A large penalty is crucial for enforcing constraints.
    max_single_dist = 0
    for i in dist:
        for j in dist[i]:
            if isfinite(dist[i][j]):
                if dist[i][j] > max_single_dist:
                    max_single_dist = dist[i][j]
    
    num_nodes = 6
    # Calculate the sum of all distances
    total_dist = sum(dist[i][j] for i in dist for j in dist[i] if isfinite(dist[i][j]))
    
    # Use a multiple of this sum as the penalty
    P = total_dist * 1# Or a higher multiple like 2.0 or 3.0
    penalty = p
    print(f"Using new penalty parameter: {P:.2f}")
    # Constraint 1: Visit each patient exactly once
    # Equation: sum_{i} x_ij = 1 for each patient j.
    # Penalty: P * (sum_i x_ij - 1)^2
    # Expanded form: P * (sum_i x_ij^2 - 2*sum_i x_ij + 2*sum_{i<k} x_ij x_kj + 1)
    # Since x^2 = x for binary variables, this becomes: P * (sum_i x_ij - 2*sum_i x_ij + 2*sum_{i<k} x_ij x_kj + 1)
    # Simplified: P * (-sum_i x_ij + 2*sum_{i<k} x_ij x_kj) + P
    # We ignore the constant P as it doesn't affect the optimization.
    # We apply this for all patient nodes (j > 0). The slides state "every vertex should have 1 edge coming from a different vertex" to visit all patients.
    for j in range(1, n): # For each patient node
        incoming_vars = [var_map[(i, j)] for i in range(n) if i != j]

        # Linear (diagonal) terms: -P * sum_i x_ij
        for idx in incoming_vars:
            Q[idx, idx] += -P
        
        # Quadratic cross terms: +2P * sum_{i<k} x_ij * x_kj
        for idx1_pos, idx1 in enumerate(incoming_vars):
            for idx2 in incoming_vars[idx1_pos + 1:]:
                a, b = (idx1, idx2) if idx1 < idx2 else (idx2, idx1)
                Q[a, b] += 2.0 * P

    # Constraint 2: Exactly 2 trips from/to the hospital
    # The slides state "We limit the number of trips to exactly 2 by setting the number of outgoing edges to 2".
    # This also implies 2 incoming edges.
    
    # 2 outgoing trips from hospital (i=0)
    # Equation: sum_{j} x_0j = 2
    # Penalty: P * (sum_j x_0j - 2)^2
    # Expanded form: P * (-3 * sum_j x_0j + 2 * sum_{j<k} x_0j x_0k) + 4P (constant ignored)
    outgoing_vars = [var_map[(0, j)] for j in range(1, n)]
    for idx in outgoing_vars:
        Q[idx, idx] += -3.0 * P
    for idx1_pos, idx1 in enumerate(outgoing_vars):
        for idx2 in outgoing_vars[idx1_pos + 1:]:
            a, b = (idx1, idx2) if idx1 < idx2 else (idx2, idx1)
            Q[a, b] += 2.0 * P
    
    # 2 incoming trips to hospital (j=0)
    # Equation: sum_{i} x_i0 = 2
    # Penalty: P * (sum_i x_i0 - 2)^2
    # Expanded form: P * (-3 * sum_i x_i0 + 2 * sum_{i<k} x_i0 x_k0) + 4P (constant ignored)
    incoming_vars = [var_map[(i, 0)] for i in range(1, n)]
    for idx in incoming_vars:
        Q[idx, idx] += -3.0 * P
    for idx1_pos, idx1 in enumerate(incoming_vars):
        for idx2 in incoming_vars[idx1_pos + 1:]:
            a, b = (idx1, idx2) if idx1 < idx2 else (idx2, idx1)
            Q[a, b] += 2.0 * P

    # Constraint 3: Enter-leave (Flow Conservation)
    # To ensure the ambulance returns to the hospital on each trip, we require each vertex be entered as many times as it is left.
    # Equation: sum_i x_ij - sum_k x_jk = 0 for each patient j.
    # Penalty: P * (sum_i x_ij - sum_k x_jk)^2
    # We apply this for all patient nodes (j > 0).
    for j in range(1, n):
        incoming_vars = [var_map[(i, j)] for i in range(n) if i != j]
        outgoing_vars = [var_map[(j, k)] for k in range(n) if k != j]

        # Linear (diagonal) terms: +P * sum_in x + +P * sum_out x - 2P * sum_in x
        for idx in incoming_vars:
            Q[idx, idx] += P
        for idx in outgoing_vars:
            Q[idx, idx] += P
        
        # Quadratic cross terms: -2P * x_in * x_out
        for idx_in in incoming_vars:
            for idx_out in outgoing_vars:
                a, b = (idx_in, idx_out) if idx_in < idx_out else (idx_out, idx_in)
                Q[a, b] += -2.0 * P

    # Constraint 4: Subtour Elimination ("No Little Loops")
    # A general subtour constraint is difficult, but the slides mention disallowing taking only 1 patient in the trip.
    # A simple way to partially achieve this is to penalize two-node loops (x_ij and x_ji).
    # Penalty: P * x_ij * x_ji
    for i in range(n):
        for j in range(i + 1, n):
            idx_ij = var_map[(i, j)]
            idx_ji = var_map[(j, i)]
            Q[idx_ij, idx_ji] += P
            Q[idx_ji, idx_ij] += P

    print(f"QUBO matrix shape: {Q.shape}")
    print(f"QUBO matrix non-zero elements: {np.count_nonzero(Q)}")

    return Q, var_map, nodes, penalty

Q_matrix, variable_map, node_list, const_penalty = create_full_qubo_matrix()


=== FULL QUBO FORMULATION (30 QUBITS) ===
Total variables (qubits): 30
Using new penalty parameter: 311.03
QUBO matrix shape: (30, 30)
QUBO matrix non-zero elements: 230


In [55]:
Q_matrix

array([[-2168.589 ,   870.8868,   870.8868,   870.8868,   870.8868,
          435.4434,     0.    ,     0.    ,     0.    ,     0.    ,
            0.    ,  1741.7736,     0.    ,     0.    ,     0.    ,
            0.    ,  1741.7736,     0.    ,     0.    ,     0.    ,
            0.    ,  1741.7736,     0.    ,     0.    ,     0.    ,
            0.    ,  1741.7736,     0.    ,     0.    ,     0.    ],
       [    0.    , -2165.721 ,   870.8868,   870.8868,   870.8868,
            0.    ,  1741.7736,     0.    ,     0.    ,     0.    ,
          435.4434,     0.    ,     0.    ,     0.    ,     0.    ,
            0.    ,     0.    ,  1741.7736,     0.    ,     0.    ,
            0.    ,     0.    ,  1741.7736,     0.    ,     0.    ,
            0.    ,     0.    ,  1741.7736,     0.    ,     0.    ],
       [    0.    ,     0.    , -2167.772 ,   870.8868,   870.8868,
            0.    ,     0.    ,  1741.7736,     0.    ,     0.    ,
            0.    ,     0.    ,  1741.7736,   

In [ ]:
# =============================================================
# SECTION 4: FULL 30-QUBIT QAOA IMPLEMENTATION
# =============================================================

def get_hamiltonian(Q):
    """Converts a QUBO matrix Q to a Qiskit Hamiltonian (SparsePauliOp)."""
    num_qubits = Q.shape[0]
    pauli_list = []
    # Off-diagonal terms: Q_ij * Z_i * Z_j
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            if not np.isclose(Q[i, j], 0.0):
                pauli_str = ['I'] * num_qubits
                pauli_str[i] = 'Z'
                pauli_str[j] = 'Z'
                # Coefficient is Q_ij/2 in standard mapping, but we combine terms
                # C = 0.5 * (Q_ij + Q_ji)
                coeff = 0.5 * (Q[i, j] + Q[j, i])
                pauli_list.append(("".join(pauli_str), coeff))

    # Diagonal terms: Q_ii * Z_i
    for i in range(num_qubits):
        if not np.isclose(Q[i, i], 0.0):
            pauli_str = ['I'] * num_qubits
            pauli_str[i] = 'Z'
            # Coefficient is -0.5 * Q_ii + constant part
            # This requires careful derivation, let's use Qiskit's tools
            # For simplicity here, we stick to the direct cost formulation
            # and let the evaluate function handle the classical energy.

    # A simpler way is to construct the operator directly for expectation value
    hamiltonian = SparsePauliOp.from_list([("I"*num_qubits, 0.0)])
    for i in range(num_qubits):
        for j in range(i, num_qubits):
            if not np.isclose(Q[i, j], 0.0):
                if i == j: # Linear term: x_i -> (I - Z_i)/2
                    coeff = Q[i,i]
                    op = SparsePauliOp.from_list([("I" * i + "Z" + "I" * (num_qubits - i - 1), -coeff/2)])
                    op += SparsePauliOp.from_list([("I" * num_qubits, coeff/2)])
                    hamiltonian += op
                else: # Quadratic term: x_i x_j -> (I - Z_i - Z_j + Z_i Z_j)/4
                    coeff = Q[i,j] + Q[j,i]
                    op = SparsePauliOp.from_list([
                        ("I" * num_qubits, coeff/4),
                        ("I"*i + "Z" + "I"*(num_qubits-i-1), -coeff/4),
                        ("I"*j + "Z" + "I"*(num_qubits-j-1), -coeff/4),
                        ("I"*i + "Z" + "I"*(j-i-1) + "Z" + "I"*(num_qubits-j-1), coeff/4)
                    ])
                    hamiltonian += op
    return hamiltonian

def create_qaoa_circuit(Q, beta, gamma, p=1):
    num_qubits = Q.shape[0]
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))

    for layer in range(p):
        # Cost unitary
        for i in range(num_qubits):
            for j in range(i + 1, num_qubits):
                if not np.isclose(Q[i, j], 0.0):
                    qc.rzz(2 * gamma[layer] * (Q[i,j] + Q[j,i]), i, j)
        for i in range(num_qubits):
             if not np.isclose(Q[i, i], 0.0):
                qc.rz(2 * gamma[layer] * Q[i, i], i)
        # Mixer unitary
        qc.rx(2 * beta[layer], range(num_qubits))

    qc.measure_all()
    return qc

def evaluate_qaoa_cost(counts, Q):
    total_cost = 0.0
    total_shots = sum(counts.values())

    for bitstring, count in counts.items():
        x = np.array([int(b) for b in bitstring[::-1]])
        cost = x.T @ Q @ x
        total_cost += cost * count

    return total_cost / total_shots if total_shots > 0 else float('inf')

def run_full_qaoa_optimization():
    print("\n=== FULL 30-QUBIT QAOA OPTIMIZATION ===")

    num_qubits = Q_matrix.shape[0]

    ## Increased shots for a more realistic simulation.
    p = 1
    shots = 1024
    print(f"QAOA depth: p={p}, Shots per evaluation: {shots}")

    # --- Simulator Setup (Including GPU Option) ---
    # To use GPU:
    # 1. Install qiskit-aer-gpu: pip install qiskit-aer-gpu
    # 2. Ensure you have a compatible NVIDIA GPU and CUDA toolkit installed.
    # 3. Uncomment the line below.
    simulator = AerSimulator(method='statevector', device='GPU')
    # simulator = AerSimulator(method='matrix_product_state', device='GPU')

    best_cost = float('inf')
    best_params = None
    # Objective function for the optimizer
    def objective_function(params):
        nonlocal best_cost, best_params
        
        # MODIFICATION: Correctly slice the params array for p > 1
        beta = params[:p]
        gamma = params[p:]

        # Create, transpile, and run the circuit
        qc = create_qaoa_circuit(Q_matrix, beta, gamma, p)
        transpiled_qc = transpile(qc, simulator)
        result = simulator.run(transpiled_qc, shots=shots).result()
        counts = result.get_counts()

        # Calculate cost
        cost = evaluate_qaoa_cost(counts, Q_matrix)

        # Track best parameters
        if cost < best_cost:
            best_cost = cost
            best_params = params
            print(f"New best cost: {cost:.2f} with params (β, γ): ({params[:p]}, {params[p:]})")

        return cost

    # Run the optimization
    print("Starting parameter optimization with COBYLA...")
    optimizer = COBYLA(maxiter=150)


    # # Run the optimization
    # print("Starting parameter optimization with Nelder-Mead...")
    # optimizer = NELDER_MEAD(maxiter=50)

    
    # initial_point = np.array([np.pi / 4, np.pi / 2]) # A common starting point
    # MODIFICATION: Correctly set initial_point for p > 1
    # For p=2, we need 4 parameters. We can repeat a common starting point.
    initial_point = np.array([np.pi / 4] * p + [np.pi / 2] * p)

    # The optimizer will find the best parameters
    opt_result = optimizer.minimize(objective_function, x0=initial_point)

    print("\nOptimization finished.")
    # Now run one final time with the best parameters found to get the final counts
    final_beta = best_params[:p]
    final_gamma = best_params[p:]
    final_qc = create_qaoa_circuit(Q_matrix, final_beta, final_gamma, p)
    final_transpiled_qc = transpile(final_qc, simulator)
    final_result = simulator.run(final_transpiled_qc, shots=shots).result()
    final_counts = final_result.get_counts()

    return {
        'cost': best_cost,
        'params': best_params,
        'counts': final_counts
    }
def decode_full_solution(bitstring, var_map, nodes):
    """Decode bitstring solution back to route"""
    print(f"\n--- Full Solution Decoding ---")
    print(f"Bitstring length: {len(bitstring)}")

    n = len(nodes)
    adjacency = np.zeros((n, n), dtype=int)

    # Convert bitstring to solution vector (little-endian convention)
    x = np.array([int(b) for b in bitstring[::-1]])

    # Only use tuple keys (i, j)
    for key, qubit_idx in var_map.items():
        if isinstance(key, tuple):  # skip string keys like "x_0_1"
            i, j = key
            if qubit_idx < len(x):
                adjacency[i, j] = x[qubit_idx]

    print("Adjacency matrix:")
    print("     ", "  ".join(f"{nodes[i]:>6}" for i in range(n)))
    for i in range(n):
        row_str = f"{nodes[i]:>6}:"
        for j in range(n):
            row_str += f"{adjacency[i, j]:>7}"
        print(row_str)

    # Extract selected edges
    edges = [(i, j) for i in range(n) for j in range(n) if adjacency[i, j] == 1]
    edge_labels = [(nodes[i], nodes[j]) for i, j in edges]
    print(f"Selected edges ({len(edges)} total): {edge_labels}")

    # Calculate distance
    total_distance = sum(dist[nodes[i]][nodes[j]] for i, j in edges)
    print(f"Total distance: {total_distance:.2f} km")

    # Constraint checks
    violations = 0
    print("\nConstraint violations:")

    # Node balance
    for j in range(n):
        incoming = sum(adjacency[i, j] for i in range(n) if i != j)
        outgoing = sum(adjacency[j, k] for k in range(n) if k != j)
        if incoming != outgoing:
            violations += 1
            print(f"  Node {nodes[j]}: in={incoming}, out={outgoing} (violation)")

    # Hospital departures
    hospital_out = sum(adjacency[0, j] for j in range(1, n))
    if hospital_out != 2:
        violations += 1
        print(f"  Hospital departures: {hospital_out} (should be 2)")

    # Patient visits
    for j in range(1, n):
        patient_visits = sum(adjacency[i, j] for i in range(n) if i != j)
        if patient_visits != 1:
            violations += 1
            print(f"  Patient {nodes[j]} visits: {patient_visits} (should be 1)")

    print(f"Total constraint violations: {violations}")

    return total_distance, violations


In [63]:

# =============================================================
# SECTION 5: RUN COMPLETE EXPERIMENT
# =============================================================

qaoa_full_results = run_full_qaoa_optimization()

# Find the most frequent bitstring from the final run
if qaoa_full_results:
    most_frequent_bitstring = max(qaoa_full_results['counts'], key=qaoa_full_results['counts'].get)
    qaoa_decoded_distance, qaoa_violations = decode_full_solution(most_frequent_bitstring, variable_map, node_list)
else:
    print("QAOA optimization failed.")
    most_frequent_bitstring = ""
    qaoa_decoded_distance, qaoa_violations = float('inf'), 999

# =============================================================
# SECTION 6: RESULTS COMPARISON AND ANALYSIS
# =============================================================

print(f"\n{'='*70}")
print("FINAL RESULTS COMPARISON")
print(f"{'='*70}")

print(f"Classical TSP Solution:")
print(f"  Best tour: {' -> '.join(classical_tour)}")
print(f"  Distance: {classical_distance:.2f} km")

if qaoa_full_results:
    print(f"\nQAOA Solution (30 qubits):")
    print(f"  Best avg. QUBO energy (cost): {qaoa_full_results['cost']:.2f}")
    print(f"  Best parameters (β, γ): ({qaoa_full_results['params'][0]:.3f}, {qaoa_full_results['params'][1]:.3f})")
    print(f"  Most frequent bitstring: {most_frequent_bitstring} (Count: {qaoa_full_results['counts'][most_frequent_bitstring]})")
    print(f"  Decoded distance of best bitstring: {qaoa_decoded_distance:.2f} km")
    print(f"  Constraint violations: {qaoa_violations}")

    if qaoa_violations == 0:
        gap = qaoa_decoded_distance - classical_distance
        gap_percent = (gap / classical_distance) * 100 if classical_distance > 0 else 0
        print(f"  Optimality gap vs classical: {gap:.2f} km ({gap_percent:.1f}%)")
    else:
        print("  Could not calculate optimality gap because the QAOA solution is not valid.")
else:
    print("\nQAOA Solution: Failed to obtain results")

# =============================================================
# SECTION 7: VISUALIZATION
# =============================================================

def create_visualization():
    print(f"\n=== VISUALIZATION ===")

    center_lat = np.mean([c[0] for c in coord_map.values()])
    center_lon = np.mean([c[1] for c in coord_map.values()])
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Add markers
    folium.Marker(location=coord_map["H"], popup="Hospital", icon=folium.Icon(color='red', icon='plus-sign')).add_to(m)
    for pid in ids:
        folium.Marker(location=coord_map[pid], popup=f"Patient {pid}", icon=folium.Icon(color='blue', icon='user')).add_to(m)

    # Add classical solution route
    if classical_tour:
        route_coords = [coord_map[node] for node in classical_tour]
        folium.PolyLine(route_coords, color='green', weight=4, opacity=0.8, popup=f"Classical Route ({classical_distance:.2f} km)").add_to(m)

    map_file = "patient_transport_map.html"
    m.save(map_file)
    print(f"Interactive map saved to {map_file}")

    # Plot QUBO matrix heatmap
    plt.figure(figsize=(10, 8))
    plt.imshow(Q_matrix, cmap='viridis', aspect='auto')
    plt.colorbar(label='QUBO Coefficient Value')
    plt.title('QUBO Matrix Heatmap')
    plt.xlabel('Variable Index')
    plt.ylabel('Variable Index')
    plt.tight_layout()
    plt.savefig('qubo_matrix_heatmap.png', dpi=150)
    plt.show()
    print("QUBO matrix heatmap saved as 'qubo_matrix_heatmap.png'")

create_visualization()


=== FULL 30-QUBIT QAOA OPTIMIZATION ===
QAOA depth: p=1, Shots per evaluation: 1024
Starting parameter optimization with COBYLA...
New best cost: 7780.58 with params (β, γ): ([0.78539816], [1.57079633])
New best cost: 7395.83 with params (β, γ): ([0.79826321], [1.47162733])
New best cost: 7299.58 with params (β, γ): ([0.86812743], [1.40008021])
New best cost: 7277.71 with params (β, γ): ([0.8717983], [1.40347503])
New best cost: 7211.11 with params (β, γ): ([0.87142908], [1.40254568])

Optimization finished.

--- Full Solution Decoding ---
Bitstring length: 30
Adjacency matrix:
           H      DT      GR      R2    R3 2      IT
     H:      0      0      1      1      0      0
    DT:      1      0      0      0      0      0
    GR:      0      0      0      1      1      1
    R2:      1      1      0      0      1      0
  R3 2:      1      1      1      0      0      1
    IT:      1      0      1      1      0      0
Selected edges (16 total): [('H', 'GR'), ('H', 'R2'), ('DT', 

KeyError: 'R3 2'